![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [395]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

from datetime import datetime

In [396]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [397]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [398]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [399]:
df_train.info()

In [400]:
df_train.head(5)

In [401]:
df_test.info()

In [402]:
df_test.head(5)

In [403]:
sample_submission.head(5)

In [404]:
sample_submission.info()

In [405]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [406]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [407]:
data.sample(5)

In [408]:
data.Reviews[1]

Как видим, отзывы состоят из 2х частей: текстовой и даты. С текстовой частью мы ничего сделать не сможем, а с датой попробуем поработать.

In [409]:
# Форматирование столбца ['Reviews'] - на список с 2-мя элементами ['отзывы текст', 'даты отзывов']
data['Reviews'] = data['Reviews'].apply(lambda s: str(s).replace('[[','[').replace(']]',']').replace("], [","]%$[").replace("'","").split(sep = '%$'))


In [410]:
# Создаем для удобства промежуточный столбец - элемент списка 'даты отзыва' 
data['Reviews_date'] = data['Reviews'].apply(lambda s: s[1] if len(s) == 2 else 'NaN') 
# В столбце ['Reviews'] оставим только отзывы
data['Reviews'] = data['Reviews'].apply(lambda s: s[0] if len(s) == 2 else 'NaN')
data.head()

In [411]:
# Функция для перевода строки в формат даты

def date_format(date_str):
    """Переводит строку в дату"""
    if len(date_str) > 10:
        dt_1 = date_str[1:11]
        dt_1 = pd.to_datetime(dt_1)
               
    if len(date_str) > 14:
        dt_2 = date_str[-11:-1]
        dt_2 = pd.to_datetime(dt_2)
          
    if len(date_str) < 10:
        return ['NaN', 'NaN']
    elif 10<len(date_str) < 22:
        return [dt_1, 'NaN']
    else:
        return [dt_1, dt_2]

In [412]:
data['date'] = data['Reviews_date'].apply(date_format)
data.drop(['Reviews_date'], axis = 1, inplace = True)
data['date_1'] = data['date'].apply(lambda d: d[0])
data['date_2'] = data['date'].apply(lambda d: d[1])
data.head()

In [413]:
current_datetime = data.date_1.max()

In [414]:
#current_datetime = datetime.now().strftime('%Y.%m.%d')
#current_datetime = datetime.strptime(current_datetime, '%Y.%m.%d')


In [415]:
data['date_1'] = (current_datetime - data['date_1']).dt.days.dropna().astype(int)
data['date_2'] = (current_datetime - data['date_2']).dt.days.dropna().astype(int)

Будем использовать эти новые созданные числовые признаки для нашей модели.

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [416]:
# Обработка пропусков в столбце Number_of_Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [417]:
data['Number_of_Reviews_isNAN']

In [418]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(data['Number of Reviews'].mean(), inplace=True)

In [419]:
#ranking_mean = df_output.groupby(['City'])['Ranking'].mean()
#restaurants_counts = data['City'].value_counts()
    #def normalize_ranking(x):
     #   return (x['Ranking'] - ranking_mean[x['City']]) / restaurants_counts[x['City']]
    
   # df_output['Ranking_normalized'] = df_output.apply(normalize_ranking, axis=1)
#restaurants_counts

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [420]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [421]:
data.head(5)

In [422]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [423]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  


In [424]:
def PriceRange(price):
    """Функция обработки 'Price Range' к виду 1 или 2 или 3"""
    
    if price == '$':
        return 1
    elif price == '$$ - $$$':
        return 2
    elif price == '$$$$':
        return 3
    else:
        return 0 

### Поработаем с restaurant_id

In [425]:
data.Restaurant_id.nunique()

In [426]:
data.Restaurant_id.value_counts()

Здесь все просто, id соответствует сети ресторанов или одному ресторану. 
Пропусков нет.

Попробуем определить, влияет ли на показатель рейтинга тот факт, является ли ресторан сетевым.

In [427]:
#Создадим датафрейм с накопленной частотой id ресторанов
rest_id = data.Restaurant_id.value_counts().reset_index()
rest_id.columns = ['Restaurant_id','number']
#Отберем те, которые встречаются более 1 раза
rest_net = rest_id[rest_id.number>1]
rest_net

In [428]:
data[data.Restaurant_id.isin(rest_net.Restaurant_id)].Rating.mean() #средний рейтинг сетевых ресторанов

In [429]:
data[~data.Restaurant_id.isin(rest_net.Restaurant_id)].Rating.mean() #средний рейтинг несетевых ресторанов

Видим, что у сетевых ресторанов средний рейтинг выше, поэтому включим в модель признак is_rest_net (является ли ресторан сетевым)

### Следующий признак - Cuisine Style


In [430]:
# создадим новую переменную для кухонь, пропуски временно заполним строкой "No data"
data['Cuisine'] = data['Cuisine Style'].fillna("['No data']")

# переведём строки в списки
data['Cuisine'] = data['Cuisine'].apply(lambda x: eval(x))

In [431]:
# Посчитаем среднее количество кухонь в ресторане
count = 0
for i in range(len(data['Cuisine'])):
    number = len(data.loc[i,'Cuisine'])
    count += number
    data.loc[i,'Cuisine count'] = number #добавим новый столбец - количество кухонь в ресторане
count/len(data['Cuisine'])

В модель будет добавлен новый признак - количество кухонь в ресторане

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [432]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [433]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [434]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [435]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Видим,что в больших городах большое смещение, поэтому будем нормализовывать переменную rating

### Посмотрим распределение целевой переменной

In [436]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [437]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [438]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [439]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

In [440]:
data.head()

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [441]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()
RANDOM_SEED = 42

In [442]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN  и прочая обработка ###################################################### 
    # Далее заполняем пропуски
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Price Range'].fillna(0, inplace=True)
    df_output['Price Range'] = df_output['Price Range'].apply(PriceRange)
    
    df_output['Reviews'] = df_output['Reviews'].apply(lambda s: str(s).replace('[[','[').replace(']]',']').replace("], [","]%$[").replace("'","").split(sep = '%$'))
    # Создаем для удобства промежуточный столбец - элемент списка 'даты отзыва' 
    df_output['Reviews_date'] = df_output['Reviews'].apply(lambda s: s[1] if len(s) == 2 else 'NaN') 
    # В столбце ['Reviews'] оставим только отзывы
    df_output['Reviews'] = df_output['Reviews'].apply(lambda s: s[0] if len(s) == 2 else 'NaN')
    
    df_output['date'] = df_output['Reviews_date'].apply(date_format)
    df_output.drop(['Reviews_date'], axis = 1, inplace = True)
    df_output['date_1'] = df_output['date'].apply(lambda d: d[0])
    df_output['date_2'] = df_output['date'].apply(lambda d: d[1])
    
    current_datetime = df_output.date_1.max()
    
    df_output['date_1'] = (current_datetime - df_output['date_1']).dt.days.dropna().astype(int)
    df_output['date_2'] = (current_datetime - df_output['date_2']).dt.days.dropna().astype(int)
    
    df_output['date_1'].fillna(df_output['date_1'].mean(), inplace=True)
    df_output['date_2'].fillna(df_output['date_2'].mean(), inplace=True)
    
    #нормализуем переменные
    # 1. Ranking
    ranking_mean = df_output.groupby(['City'])['Ranking'].mean()
    restaurants_counts = df_output['City'].value_counts()
    def normalize_ranking(x):
        return (x['Ranking'] - ranking_mean[x['City']]) / restaurants_counts[x['City']]
    
    df_output['Ranking_normalized'] = df_output.apply(normalize_ranking, axis=1)
    #df_output.drop(['Ranking'], axis = 1, inplace = True)
    
    # 2. Reviews 
    
    reviews_mean = df_output.groupby(['City'])['Number of Reviews'].mean()
    def normalize_reviews(x):
        return (x['Number of Reviews'] - reviews_mean[x['City']]) / restaurants_counts[x['City']]
    
    df_output['Number of Reviews normalized'] = df_output.apply(normalize_reviews, axis=1)
    #df_output.drop(['Number of Reviews'], axis = 1, inplace = True)
    
    # ################## 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    #df_output = pd.get_dummies(df_output, columns=['Price Range'], dummy_na=True) #результат по МАЕ был хуже

    
    
    
    
    # ################### 4. Feature Engineering ####################################################
    
    # ### 1. Создание новой переменной - количество кухонь в ресторане ###
    
    # создадим новую переменную для кухонь, пропуски временно заполним строкой "No data"
    df_output['Cuisine'] = df_output['Cuisine Style'].fillna("['No data']")

    # переведём строки в списки
    df_output['Cuisine'] = df_output['Cuisine'].apply(lambda x: eval(x))
    
    
    # Найдем количество кухонь в каждом ресторане
    count = 0
    for i in range(len(df_output['Cuisine'])):
        number = len(df_output.loc[i,'Cuisine'])
        count += number
        df_output.loc[i,'Cuisine count'] = number #добавим новый столбец - количество кухонь в ресторане
    
    data_new = df_output['Cuisine'].explode()
    data_new = set(data_new) # выделим все названия кухонь и создадим новые столбцы
    for element in data_new:
        df_output[element] = 0
    
    for i in range(len(df_output)):
        for element in df_output.iloc[i]['Cuisine']:
            df_output.loc[i, element] = 1 #попытаемся сделать что-то аналогичное One-Hot Encoding 

    # ### 2. Создание новой переменной - является ли ресторан сетевым ###
   

    # Cоздадим датафрейм с накопленной частотой id ресторанов
    rest_id = df_output['Restaurant_id'].value_counts().reset_index()
    rest_id.columns = ['Restaurant_id','number']
    
    rest_id_dict = rest_id.set_index('Restaurant_id').T.to_dict('int').get('number') #преобразуем в словарь для ускорения работы
    id_list = list(df_output['Restaurant_id'])
    
    for i in range(len(id_list)):
        id_ = id_list[i]
        number_ = rest_id_dict.get(id_)
        if number_ and (number_ > 1):
            df_output.loc[i, 'is_rest_net'] = 1 # значение 1 для сетевых ресторанов
        else:
            df_output.loc[i, 'is_rest_net'] = 0 # значение 0 для несетевых ресторанов
    
   
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [443]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [444]:
df_preproc.info()

In [445]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [446]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [447]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [448]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [449]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [450]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [451]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [452]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [453]:
test_data.sample(10)

In [454]:
test_data = test_data.drop(['Rating'], axis=1)

In [455]:
sample_submission

In [456]:
predict_submission = model.predict(test_data)

In [457]:
predict_submission

In [458]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
